# What are the demographic characteristics of neighborhoods where entitlements are?

In [22]:
import geopandas
import intake
import IPython.display
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import pandas
import pcts_census_utils

cat = intake.open_catalog("../catalogs/*.yml")

In [2]:
# Load data

prefix_list = pcts_census_utils.FULL_PREFIX_LIST
suffix_list = pcts_census_utils.FULL_SUFFIX_LIST

remove_prefix = ["ENV"]
remove_suffix = [
    "EIR",
    "IPRO",
    "CA",
    "CATEX",
    "CPIO",
    "CPU",
    "FH",
    "G",
    "HD",
    "HPOZ",
    "ICO",
    "K",
    "LCP",
    "NSO",
    "S",
    "SN",
    "SP",
    "ZAI"
]

prefix_list = [x for x in prefix_list if x not in remove_prefix]
suffix_list = [x for x in suffix_list if x not in remove_suffix]

In [3]:
# Load PCTS and subset to the prefix / suffix list we want
pcts = pcts_census_utils.subset_pcts(
    prefix_list=prefix_list,
    suffix_list=suffix_list,
    get_dummies=True,
    verbose=True,
)
pcts = pcts_census_utils.drop_child_cases(pcts, keep_child_entitlements=True)

Parsing PCTS case numbers
Getting dummy indicators for case types
Prefixes with no associated cases:  {'TT', 'HPO', 'VTT', 'PS'}
Suffixes with no associated cases:  {'REC4', 'EAF', 'SUP1', 'ADD1', 'REC3', 'SCPE', 'SE', 'REC5', 'ND', 'SCEA', 'REC2'}


In [4]:
# ACS data for income, race, commute, tenure
census = cat.census_analysis_table.read()

# Census tracts
tracts = cat.census_tracts.read()
tracts = (
    tracts[["GEOID10", "geometry"]]
    .rename(columns = {"GEOID10": "GEOID"})
)

In [5]:
# Crosswalk linking AIN to tract GEOID
parcel_to_tract = cat.crosswalk_parcels_tracts.read()
parcel_to_tract = parcel_to_tract[["AIN", "num_AIN", "GEOID"]]

In [6]:
# Merge entitlements with tract using crosswalk
pcts = pandas.merge(
    pcts,
    parcel_to_tract, 
    on="AIN",
    how="inner",
    validate="m:1",
)

In [7]:
#  Clean AIN data and get rid of outliers
case_counts = pcts.CASE_ID.value_counts()
big_cases = pcts[pcts.CASE_ID.isin(case_counts[case_counts > 20].index)]

pcts = pcts[~pcts.CASE_ID.isin(big_cases.CASE_ID)]

In [8]:
# Our first pass at analyzing entitlements is to count the number
# of cases for each census tract, to see which kinds of entitlements
# are being applied for in which types of census tract:
entitlement_counts = (pcts
    [["GEOID", "CASE_YR_NBR"] + suffix_list]
    .astype({c: "int64" for c in suffix_list})
    .groupby(["GEOID", "CASE_YR_NBR"])
    .sum()
).reset_index(level=1).rename(columns={"CASE_YR_NBR": "year"})
entitlement_counts = entitlement_counts.assign(
    year=entitlement_counts.year.astype("int64")
)

In [9]:
# Merge the census data with the entitlements counts:
joined = pandas.merge(
    census,
    entitlement_counts,
    on="GEOID",
    how="left", 
    validate="1:m"
).dropna().sort_values(["GEOID", "year"]).astype(
    {c: "int64" for c in suffix_list}
)

In [10]:
# Plot entitlement stats against median household income,
# population density, and geography:
def plot_entitlement(df, tracts, suffix, year="2017"):
        
    if year == "all":
        to_plot = df[(df[suffix] != 0) & (df.year >= 2010)]
        to_plot = to_plot.groupby("GEOID").agg({
            suffix: "sum",
            "medhhincome": "first",
            "density": "first"
        }).reset_index()
    else:
        to_plot = df[(df[suffix] != 0) & (df.year == int(year))]
    
    # Merge in geometry
    final_df = tracts.merge(
        to_plot,
        on="GEOID", 
        how="left",
    ).fillna(
        {suffix: 0}
    )
    
    fig = plt.figure(figsize=(12,8))
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=1)
    ax1.set_xlabel("Median household income")
    ax1.set_ylabel("Entitlement count per tract")
    ax1.scatter(final_df.medhhincome, final_df[suffix])

    ax2 = plt.subplot2grid((2, 2), (1, 0), colspan=1)
    ax2.set_xlabel("Population density")
    ax2.set_ylabel("Entitlement count per tract")
    ax2.scatter(final_df.density, final_df[suffix])
    
    ax3 = plt.subplot2grid((2, 2), (0, 1), rowspan=2)
    ax3.axis("off")
    
    final_df.plot(
        ax=ax3,
        column=suffix,
        cmap="viridis",
    )
    
    plt.close() # Prevent double plotting
    return fig

In [11]:
years = [("All (from 2010 - 2019)", "all")] + [(str(i), str(i)) for i in range(2010, 2020)]
year_dropdown = ipywidgets.Dropdown(description="Year", options=years)
suffix_dropdown = ipywidgets.Dropdown(description="Suffix")
output = ipywidgets.Output()

display(year_dropdown)
display(suffix_dropdown)
display(output)

change_guard = False

def on_suffix_selection(*args):
    global change_guard
    if change_guard:
        return
    output.clear_output(wait=True)
    suffix = suffix_dropdown.value
    year = year_dropdown.value
    with output:
        display(plot_entitlement(joined, tracts, suffix, year))

def on_year_selection(*args):
    global change_guard
    if year_dropdown.value == "all":
        condition = (joined.year >= 2010)
    else:
        condition = (joined.year == int(year_dropdown.value))
    counts = joined.loc[condition, suffix_list].sum()
    # Sort by alphabetical or in descending value of counts?
    #counts = counts.sort_values(ascending=False)
    old_val = suffix_dropdown.value 
    change_guard=True
    suffix_dropdown.options = [
        (f"{name} ({count:,} applications)", name) 
        for name,count in zip(counts.index, counts)
    ]
    if old_val in counts.index:
        suffix_dropdown.value = old_val
    else:
        suffix_dropdown.index = 0
    change_guard=False
    on_suffix_selection()

on_year_selection()
suffix_dropdown.observe(on_suffix_selection, names="value")
year_dropdown.observe(on_year_selection, names="value")

Dropdown(description='Year', options=(('All (from 2010 - 2019)', 'all'), ('2010', '2010'), ('2011', '2011'), (…

Dropdown(description='Suffix', options=(), value=None)

Output()

## What are the outliers?

In [126]:

case_dropdown = ipywidgets.Dropdown(
    description="Outlier cases",
    options=tuple(
        (f"{r[1]} ({r[2]} parcels)", r[0]) for r in
        big_cases.groupby("CASE_ID").agg(
            {"CASE_NBR": "first", "AIN": "count"}
        ).sort_values("AIN", ascending=False).itertuples()
    )
)

output = ipywidgets.Output()

display(case_dropdown)
display(output)


def plot_case(case_id):
    to_map = geopandas.GeoDataFrame(
        big_cases[big_cases.CASE_ID == case_id].groupby("GEOID").agg({
            "AIN": "count",
            "PROJ_DESC_TXT": "first"
        }).merge(
            tracts,
            how="right",
            left_index=True,
            right_on="GEOID"
        )
    )
    description = to_map.PROJ_DESC_TXT.dropna().iloc[0]
    with output:
        output.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12,12))
        to_map.fillna(0).plot(column="AIN",legend=True, ax=ax)
        plt.close()
        print(description)
        display(fig)

def on_case_selection(*args):
    plot_case(case_dropdown.value)

case_dropdown.observe(on_case_selection, names=["value"])
on_case_selection()

Dropdown(description='Outlier cases', options=(('CPC-2010-2278-GPA (10159 parcels)', 180067.0), ('CPC-2010-589…

Output()

In [ ]:
big_cases.groupby("CASE_ID").agg({
    "CASE_NBR": "first",
    "PROJ_DESC_TXT": "first",
    "AIN": "count"
}).sort_values("AIN", ascending=False).rename(
    columns={"PROJ_DESC_TXT": "Description", "AIN": "Parcels"}
).style